In [1]:
import aerosandbox as asb
from aerosandbox.atmosphere import Atmosphere as atmo
from aerosandbox.library import winds as lib_winds
import aerosandbox.numpy as np

In [2]:
##### Initialize Optimization
opti = asb.Opti(  # Normal mode - Design Optimization
    cache_filename="solution.json",
    save_to_cache_on_solve=True
)
# opti = asb.Opti( # Alternate mode - Frozen Design Optimization
#     variable_categories_to_freeze=["des"],
#     cache_filename="cache/optimization_solution.json",
#     load_frozen_variables_from_cache=True,
#     ignore_violated_parametric_constraints=True
# )

### Maximize endurance
minimize = "power"    # "eval-able" expression

### Configurations
make_plots = True
allow_trajectory_optimization = True
climb_opt = False
variable_pitch = False
use_propulsion_fits_from_FL2020_1682_undergrads = True
n_timesteps_per_segment = 180    # simulation parameters
min_cruise_altitude = 10         # [m]
max_cruise_altitude = 100        # [m]
ultimate_load_factor = 3.0
electrical_load_limit = 6        # C rating for typical li-ion battery

In [3]:
### Operating Parameters
##### Initialize design optimization variables (all units in base SI or derived units)

day_of_year = opti.parameter(value=244)  # Julian day. June 1 is 153, June 22 is 174, Aug. 31 is 244
latitude = opti.parameter(value=22)      # [deg]
# altitude = opti.variable(init_guess=95, lower_bound=min_cruise_altitude, upper_bound=max_cruise_altitude) # log_transform=True)
altitude = opti.parameter(value=100)    # [m]

##### Atmosphere
my_atmosphere = atmo(altitude=altitude)
P = my_atmosphere.pressure()
rho = my_atmosphere.density()
T = my_atmosphere.temperature()
mu = my_atmosphere.dynamic_viscosity()
a = my_atmosphere.speed_of_sound()
g = 9.81  # gravitational acceleration [m/s^2]

def wind_speed_func(alt):
    day_array = np.full(shape=alt.shape[0], fill_value=1) * day_of_year
    latitude_array = np.full(shape=alt.shape[0], fill_value=1) * latitude
    speed_func = lib_winds.wind_speed_world_95(alt, latitude_array, day_array)
    return speed_func


mass_payload = 0.2      # mass of imaging system [kg]
mass_equipment = 0.3    # mass of sensors, fmu, single-board computer [kg]
mass_propulsion = 0.3   # mass of esc, motor and propellors [kg]
num_batt_cell = opti.parameter(value=2)     # number of cells per battery pack (16.4V - 12V)
mass_per_battery_cell = 0.047               # mass of a battery cell [kg]
mass_battery_pack = 0.047 * num_batt_cell   # mass of a single li-ion 4-cell pack [kg]
battery_nominal_voltage = 3.7               # [V]
capacity_per_battery_pack = 3.7             # [Ah]
energy_per_battery_pack = capacity_per_battery_pack * battery_nominal_voltage * 3600      # cell capacity Ah * 3.7V * conversion factor  [J]

##### Margins
electrical_to_mechanical_efficiency = opti.parameter(value=0.8)
allowable_battery_depth_of_discharge = opti.parameter(
    value=0.85)    # How much of the battery can you actually use? # Reviewed w/ Annick & Bjarni 4/30/2020


In [4]:
### Design Optimization Variables

airspeed = opti.variable(
    init_guess=20,
    scale=1e-2,
    category="ops",
    log_transform=True
)

# opti.subject_to(airspeed < 75 / 3.6)

thrust_force = opti.variable(
    init_guess=15,
    category="ops",
    log_transform=True
)

flight_time = opti.variable(init_guess=40*60, lower_bound=30*60)    # [s]
power = thrust_force * airspeed                                     # [W]
required_elec_power = power / electrical_to_mechanical_efficiency   # [W]
energy = power * flight_time                                        # [J]

#### Battery model
# num_battery_packs = opti.parameter(value=1)
num_battery_packs = opti.variable(init_guess=1, scale=1)
opti.subject_to(num_battery_packs >= 1)

num_cells_per_battery = opti.variable(init_guess=2)
opti.subject_to([
    num_cells_per_battery >= 2,    # ESC requirement
    (electrical_load_limit * capacity_per_battery_pack) * 0.7 > required_elec_power / (battery_nominal_voltage * num_cells_per_battery)
])

opti.subject_to(energy < (num_battery_packs * energy_per_battery_pack) *
                          electrical_to_mechanical_efficiency *
                          allowable_battery_depth_of_discharge)

mass_batteries = num_battery_packs * num_cells_per_battery * mass_per_battery_cell


#### Aerodynamic performance
wing_area = opti.variable(init_guess=0.4, log_transform=True, category="des")    # [m^2]
wing_span = opti.variable(init_guess=0.8, log_transform=True, category="des")    # [m]
opti.subject_to(wing_span < 1)

aspect_ratio = wing_span ** 2 / wing_area
oswald_efficiency = opti.parameter(value=0.86)
airfoil_thickness_fraction = 0.12                  # airfoil thickness to chord ratio [-]

# mass_total_empirical = opti.variable(init_guess=3, log_transform=True)
# mass_total_empirical = ( mass_total_empirical * g / wing_area) ** 3 * 9.6e-6    # Design of Solar Powered Airplanes for Continuous Flight
# mass_total_empirical = ( wing_area - 1.1585) / 1.042e-2    # Empirical Correlations for Geometry Build-Up of Fixed Wing Unmanned Air Vehicles
# mass_total_empirical = (6.25 * wing_area) ** 1.5    # Assembly and Initial Analysis of a Database p.16 (Aerospace Division, Defence Science and Technology Organisation, DoD, Australian Government)
mass_total_empirical = (7.8 * wing_area) ** 1.5    # Assembly and Initial Analysis of a Database p.20 (Wing Loading)


#### Mass model
mass_total = opti.variable(init_guess=2)    # [kg]

# Structural limitation
opti.subject_to(mass_total * g / wing_area < 155)

# Fuselage model
payload_mass_fraction = 0.21    # Assembly and Initial Analysis of a Database p.45 (useful-load-mass function)
mass_fuselage = ( mass_batteries + mass_equipment + mass_payload ) / 0.36

# Wing weight model (https://github.com/peterdsharpe/AeroSandbox/blob/4d3f92ce5f183ff1289ca3b42365ce936e072452/tutorial/02%20-%20Design%20Optimization/02%20-%20Wing%20Drag%20Minimization,%20with%20practical%20considerations.ipynb)
W_W_coeff1 = 8.71e-5  # Wing Weight Coefficient 1 [1/m]
W_W_coeff2 = 45.24  # Wing Weight Coefficient 2 [Pa]
weight_wing_structural = W_W_coeff1 * (
        ultimate_load_factor * aspect_ratio ** 1.5 *
        (mass_fuselage * g * mass_total * g * wing_area) ** 0.5
) / airfoil_thickness_fraction
weight_wing_surface = W_W_coeff2 * wing_area
mass_wing = (weight_wing_surface + weight_wing_structural) / g

opti.subject_to([
    mass_wing > mass_total_empirical * 0.2,
    mass_wing < mass_total
])

opti.subject_to([
    mass_total > mass_wing + mass_fuselage + mass_propulsion,
    mass_total < 7    # HKCAD UAS CAT A2
])

C_L_cruise = opti.variable(init_guess=1.0, scale=1e-1, lower_bound=0.2, upper_bound=1.0)


q = 1 / 2 * rho * airspeed ** 2
lift_force = q * wing_area * C_L_cruise
C_D0 = opti.parameter(value=0.03)
drag_force = q * wing_area * (C_D0 + C_L_cruise **2 / (3.141592654 * oswald_efficiency * aspect_ratio))

#### SLUF condition
force_z = lift_force - (mass_total * g)
force_x = thrust_force - drag_force
opti.subject_to([
    lift_force >= mass_total * g,
    thrust_force >= drag_force,
    force_z < 1,
    force_x < 1
])

#### Hand launch capability
airspeed_stall = (2 * mass_total * g / ( 1.23 * wing_area * 1.2 )) ** 0.5
airspeed_TO = 1.2 * airspeed_stall

# opti.subject_to([
#     airspeed_TO < 14,
#     airspeed_TO > 1.2 * airspeed_stall
# ])    # assume I can throw the plane out at 30 km/h


In [5]:
##### Add objective
objective = eval(minimize)


##### Add tippers
things_to_slightly_minimize = 0

for tipper_input in [
    wing_span / 50,
    energy_per_battery_pack * num_battery_packs / 3600 / 5e4,
    # (hour[-1] - hour[0]) / 24,
]:
    try:
        things_to_slightly_minimize += tipper_input
    except NameError:
        pass

# Dewiggle
penalty = 0

# for penalty_input in [
#     thrust_force / 10,
#     # net_accel_parallel / 1e-1,
#     # net_accel_perpendicular / 1e-1,
#     airspeed / 2,
#     # flight_path_angle / 2,
#     # alpha / 1,
# ]:
#     penalty += np.sum(np.diff(np.diff(penalty_input)) ** 2) / n_timesteps_per_segment

opti.minimize(
    objective
    # + penalty
    # + 1e-3 * things_to_slightly_minimize
)

In [6]:
%%capture output

sol = opti.solve(
    max_iter=10000,
    options={
        "ipopt.max_cpu_time": 600
    }
)

# Print a warning if the penalty term is unreasonably high
penalty_objective_ratio = np.abs(sol.value(penalty / objective))
if penalty_objective_ratio > 0.01:
    print(
        f"\nWARNING: High penalty term, non-negligible integration error likely! P/O = {penalty_objective_ratio}\n")

In [7]:
opti.debug.value(num_cells_per_battery)
print(opti.debug.value(thrust_force/mass_total/g))    # extremely underpowered
print(opti.debug.value(mass_wing/mass_total))
print(opti.debug.value(mass_fuselage/mass_total))

0.12059671368472095
0.2988739897898685
0.6238118139525636


In [8]:
print(f"Flight time: {opti.debug.value(flight_time) / 60:1.2f} minutes" )
print(f"Total mass: {opti.debug.value(mass_total):1.2f} kg")
print(f"Wing mass: {opti.debug.value(mass_wing):1.2f} kg")
print(f"Wing loading: {opti.debug.value(mass_total * g / wing_area):1.2f} N/m^2")
print(f"Wing area: {opti.debug.value(wing_area):1.2f} m^2")
print(f"Wing span: {opti.debug.value(wing_span):1.2f} m")
print(f"AR: {opti.debug.value(aspect_ratio):1.2f}")    # aiming for > 8
print(f"RE: {opti.debug.value(airspeed * wing_area / wing_span / mu):1.0f}")
print(f"C_L_cruise: {opti.debug.value(C_L_cruise):1.2f}")
print(f"C_D0: {opti.debug.value(C_D0):1.2f}")
print(f"TAS_CRUISE: {opti.debug.value(airspeed * 3.6):1.4f} km/h")
print(f"TAS_TO: {opti.debug.value(airspeed_TO * 3.6):1.4f} km/h")
print(f"Alt: {opti.debug.value(altitude):1.2f} m")
print(f"PWR: {opti.debug.value(power):1.2f} W")
print(f"Battery config: {opti.debug.value(num_cells_per_battery):1.0f}C{opti.debug.value(num_battery_packs):1.0f}P")

Flight time: 30.00 minutes
Total mass: 3.88 kg
Wing mass: 1.16 kg
Wing loading: 155.00 N/m^2
Wing area: 0.25 m^2
Wing span: 1.00 m
AR: 4.07
RE: 220307
C_L_cruise: 1.00
C_D0: 0.03
TAS_CRUISE: 57.6891 km/h
TAS_TO: 62.6068 km/h
Alt: 100.00 m
PWR: 73.56 W
Battery config: 2C4P
